## Twitter NLP


In [21]:
import json, os, string
import platform
from collections import Counter
import itertools as it
from twitter_client import get_twitter_client
from tweepy import Cursor
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from selectfile import FileBrowser
import spacy


In [22]:
client = get_twitter_client()
#print(client.me())

## Getting the data using Twitter API

In [23]:
#twitter_user = input('Please introduce the twitter user name')

In [24]:
# print(twitter_user)
# cwd = os.getcwd()
# if os.path.isdir(os.path.join(cwd,'raw_data')):
#     fname = os.path.join(cwd, 'raw_data', 'user_timeline_{}.jsonl'.format(twitter_user)) 
# else:
#     fname = os.path.join(os.mkdir(os.path.join(cwd,'raw_data')), 'user_timeline_{}.jsonl'.format(twitter_user ))   
    
# print(fname)

In [25]:
# with open(fname, 'w') as f:
#     for page in Cursor(client.user_timeline, screen_name=twitter_user, count=200).pages(16):
#         for status in page:
#             f.write(json.dumps(status._json) + "\n")
   

In [31]:
raw_data_file = FileBrowser()
raw_data_file.widget()

In [32]:
raw_data_file
raw_data_file.path

'C:\\Users\\Rodolfo\\Desktop\\NLP\\raw_data\\user_timeline_jeremycorbyn.jsonl'

## Tokenization

In [33]:
def process(tokenizer, text, stopwords=[]):
    """Process the text of a tweet:
    - Lowercase
    - Tokenize
    - Stopword removal
    - Digits removal
    Return: list of strings
    """
    text = text.lower()
    tokens = tokenizer(text)
    #print(tokens)
    return [tok for tok in tokens if tok not in stopwords and not tok.isdigit()]

In [34]:
tweet_tokenizer = TweetTokenizer()
punct = list(string.punctuation)
stopword_list = stopwords.words('english') + punct + ['rt', 'via', '...']
tf = Counter()
with open(raw_data_file.path, 'r') as f:
    for line in f:
        tweet = json.loads(line)
        tokens = process(tweet_tokenizer.tokenize, text=tweet['text'], stopwords=stopword_list)
        tf.update(tokens)
for tag, count in tf.most_common(20):
    print("{}: {}".format(tag, count))

…: 2295
labour: 728
’: 433
today: 429
people: 330
government: 313
@uklabour: 300
tories: 247
nhs: 210
@theresa_may: 199
country: 166
us: 166
great: 153
tory: 144
vote: 143
#forthemany: 131
must: 130
years: 129
campaign: 127
many: 115


## Using spaCy

In [35]:
nlp = spacy.load('en')

In [36]:
nlp.pipeline

[('tagger', <spacy.pipeline.Tagger at 0x2130868be0>),
 ('parser', <spacy.pipeline.DependencyParser at 0x213090a990>),
 ('ner', <spacy.pipeline.EntityRecognizer at 0x213090aba0>)]

##  spaCy sentences contained in each tweet

In [62]:
tf = Counter()
with open(raw_data_file.path, 'r') as f:
    sample = it.islice(f, 1, 10)
    #for line in f:
    for num, line in enumerate(sample):
        tweet = json.loads(line)
        print('\n', 'Twitter .-', num, nlp(tweet['text']), '\n')
        for num, sent in enumerate(nlp(tweet['text']).sents):
            print('Sentence .-', num, sent)
        #print( [tok for tok in nlp(tweet['text'])], '\n')
#         tokens = process(nlp, text=tweet['text'], stopwords=stopword_list)
#         tf.update(tokens)
# for tag, count in tf.most_common(20):
#     print("{}: {}".format(tag, count))


 Twitter .- 0 RT @RLong_Bailey: Labour's position on #fracking is unequivocal: we will ban it. I have recently visited communities opposed to fracking in… 

Sentence .- 0 RT @RLong_Bailey:
Sentence .- 1 Labour's position on #fracking is unequivocal: we will ban it.
Sentence .- 2 I have recently visited communities opposed to fracking in…

 Twitter .- 1 Businesses creating budget private schools is no way to solve the education crisis - in fact it will only make it w… https://t.co/23MUBEcb7C 

Sentence .- 0 Businesses creating budget private schools is no way to solve the education crisis - in fact it will only make it w… https://t.co/23MUBEcb7C

 Twitter .- 2 Tory rail mayhem has gone on too long. https://t.co/vEVZpbkM6O 

Sentence .- 0 Tory rail mayhem has gone on too long.
Sentence .- 1 https://t.co/vEVZpbkM6O

 Twitter .- 3 Ten years ago, @JohnMcdonnellMP said the banks needed to work for the many not the few.

He was right then and is n… https://t.co/oKaXsIsqoV 

Sentence .- 0 Ten

##  spaCy entity recognition contained in each tweet

In [66]:
with open(raw_data_file.path, 'r') as f:
    sample = it.islice(f, 1, 10)
    #for line in f:
    for num, line in enumerate(sample):
        tweet = json.loads(line)
        print('\n', 'Twitter .-', num, nlp(tweet['text']), '\n')
        for num, ent in enumerate(nlp(tweet['text']).ents):
            print('Entity .-', num, ent, '-', ent.label_)


 Twitter .- 0 RT @RLong_Bailey: Labour's position on #fracking is unequivocal: we will ban it. I have recently visited communities opposed to fracking in… 

Entity .- 0 RT @RLong_Bailey - ORG
Entity .- 1 Labour - ORG
Entity .- 2 # - ORG

 Twitter .- 1 Businesses creating budget private schools is no way to solve the education crisis - in fact it will only make it w… https://t.co/23MUBEcb7C 


 Twitter .- 2 Tory rail mayhem has gone on too long. https://t.co/vEVZpbkM6O 


 Twitter .- 3 Ten years ago, @JohnMcdonnellMP said the banks needed to work for the many not the few.

He was right then and is n… https://t.co/oKaXsIsqoV 

Entity .- 0 Ten years ago - DATE
Entity .- 1 @JohnMcdonnellMP - ORG

 Twitter .- 4 RT @JonAshworth: More news today on the awful consequences of huge staffing gaps across our NHS. Utterly unacceptable cancer patients force… 

Entity .- 0 RT - ORG
Entity .- 1 today - DATE
Entity .- 2 NHS - ORG

 Twitter .- 5 RT @RLong_Bailey: Our High Streets are dying and the Tori